Reads in a geopackage and returns a metadata csv

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings
from shapely import box

In [ ]:
# filter out warnings to prevent verbose output
warnings.filterwarnings("ignore", message=".*Measured \\(M\\) geometry types are not supported.*")

# path to the geopackage 
geopackage = 'path/to/geopackage.gpkg'

In [ ]:
# retrieves a list of layers in the geopackage as layer_list
pkg_layers = gpd.list_layers(geopackage)
layer_list = pkg_layers['name'].values.tolist()

In [ ]:
# create a dataframe of values for each layer in the geopackage with bounds, geometry and geometry type
# remove invalid geometry layers and reindex
lbounds, lgeom, lgt, gtlist = list(), list(), list(), list()

for i in layer_list:
    layer = gpd.read_file(geopackage, layer=i)
    layer_bounds = layer.geometry.total_bounds
    lbounds.append(layer_bounds)
    geom = box(*layer.geometry.total_bounds)
    lgeom.append(geom)
    gt = layer.geom_type.unique()
    gtlist.append(gt)
gpkgdf = pd.DataFrame({'name':layer_list,'bounds':lbounds,'geometry':lgeom,'geomtype':gtlist})
gpkgdf = gpd.GeoDataFrame(gpkgdf, crs="EPSG:27700")
gpkgdf = gpkgdf.loc[gpkgdf['geometry'].is_valid, :]
gpkgdf.reset_index(drop=True, inplace=True)
gpkgdf.head(2)

In [ ]:
# if necessary create an amended geometry column via conversion, in this case to EPSG:4326, but check QGIS/ARCGIS 
gpkgdf['epsg4326'] = gpkgdf['geometry'].to_crs("EPSG:4326")
gpkgdf.head(2)

In [ ]:
# get the bounding box directional values
def getbb(bvalue,dir):
    if np.isnan(bvalue):
        res = None
    else:
        bvint = int(bvalue*10000)
        vstring =str(bvint).replace('-', '0')
        while len(vstring) < 7:
            vstring = '0'+vstring
        if bvalue >= 0:
            if dir == 'WE':
                res = 'E'+vstring
            else:
                res = 'N'+vstring
        else:
            if dir == 'WE':
                res = 'W'+vstring
            else:
                res = 'S'+vstring
    return res  

# formats boundary as degrees minutes seconds eg N0557934 to N055°79'34"
def formatCoords (boundary):
    return boundary[:4] + '°' + boundary[4:6] + '\'' + boundary[6:8] + '"'

In [ ]:
# create a dataframe of bounding box values for each layer in the geopackage with valid geometry
# use geometry column if no adjustment was needed to projection
bbw, bbs, bbe, bbn, c255  = list(), list(), list(), list(), list()
 
gvals = gpkgdf['epsg4326'].values.tolist()
for i in gvals:
    bounds = i.bounds
    wval = getbb(bounds[0],'WE')
    sval = getbb(bounds[1],'NS')
    eval = getbb(bounds[2],'WE')
    nval = getbb(bounds[3],'NS')
    cval = '(%s--%s/%s--%s)' % (formatCoords(wval), formatCoords(eval), formatCoords(nval), formatCoords(sval))
    bbw.append(wval)
    bbs.append(sval)
    bbe.append(eval)
    bbn.append(nval)
    c255.append(cval)
bbdf = pd.DataFrame({'Bounding_Box_W':bbw, 'Bounding_Box_E':bbe, 'Bounding_Box_N':bbn, 'Bounding_Box_S':bbs,'Bounding_Box_255':c255})
bbdf.head(2)

In [ ]:
# joins geometry to bounding box dataframe
# outputs to csv 
meta = gpkgdf.join(bbdf)
meta.to_csv('gpkg_meta.csv')